In [2]:
import os
import geopandas as gpd

# os.environ["USE_PYGEOS"] = os.environ.get("USE_PYGEOS", "0")
local_crs = 32636
# path to data
example_data_path = "./data"

## Input data fetching
The input "blocks" data also should be obtained from the previous example result

In [3]:
from masterplan_tools.preprocessing import DataGetter, AggregateParameters

blocks = gpd.read_parquet(os.path.join(example_data_path, "etalon_blocks.parquet")).to_crs(local_crs)
getter = DataGetter(blocks=blocks)

## City blocks information aggregating
Here we aggregate information about buildings, greenings and parkings into the city blocks

In [4]:
buildings = gpd.read_file(os.path.join(example_data_path, "houses.geojson"))

In [5]:
buildings.to_crs(local_crs, inplace=True)

In [6]:
# buildings[['building_area', 'living_area', 'storeys_count', 'living_area_pyatno', 'total_area']] = 0
# buildings['is_living'] = False
# buildings['geometry'] = buildings['geometry'].centroid
buildings.drop(labels=['name', 'address', 'municipality', 'ukname'], axis=1, inplace=True)

In [7]:
buildings['living_area_pyatno'] = 0
buildings['total_area'] = 0
buildings['storeys_count'] = 0

In [10]:
import math
buildings['population_balanced'] = buildings['population_balanced'].fillna(0).apply(math.ceil)

In [14]:
buildings['is_living'] = buildings['population_balanced']>0

In [15]:
# buildings = gpd.read_parquet(os.path.join(example_data_path, "buildings.parquet"))
greenings = gpd.read_parquet(os.path.join(example_data_path, "greenings.parquet")).rename_geometry('geometry')
parkings = gpd.read_parquet(os.path.join(example_data_path, "parkings.parquet")).rename_geometry('geometry')

aggr_params = AggregateParameters(
  buildings=buildings,
  greenings=greenings,
  parkings=parkings
)

aggregated_blocks = getter.aggregate_blocks_info(params=aggr_params)
aggregated_blocks.to_gdf().head()

Restoring living area:   0%|          | 0/89164 [00:00<?, ?it/s]

Restoring living area squash:   0%|          | 0/89164 [00:00<?, ?it/s]

,geometry,landuse,block_id,is_living,current_population,floors,current_living_area,current_green_capacity,current_green_area,current_parking_capacity,current_industrial_area,area
0,"POLYGON ((303022.702 6675775.060, 303022.795 6...",no_dev_area,0,False,0.0,NaN,0.000000,0.0,0.0,0.0,0.0000,4.399754e-01
1,"POLYGON ((303012.608 6675801.562, 303012.586 6...",no_dev_area,1,False,0.0,NaN,0.000000,0.0,0.0,0.0,0.0000,3.251122e-02
2,"POLYGON ((303064.339 6675806.849, 303064.442 6...",no_dev_area,2,False,0.0,NaN,0.000000,0.0,0.0,0.0,0.0000,6.694614e+00
3,"POLYGON ((303428.643 6675829.284, 303425.776 6...",buildings,3,True,2376.0,0.0,10776.770592,0.0,0.0,0.0,14483.6073,5.081055e+06
4,"POLYGON ((303429.370 6675800.664, 303429.236 6...",no_dev_area,4,False,0.0,NaN,0.000000,0.0,0.0,0.0,0.0000,7.489756e+02


## Accessibility matrix 
Accessibility matrix is generated using `CityGeoTools` library graphml 

In [14]:
import networkx as nx

#we get this from CityGeoTools
transport_graph = nx.read_graphml(os.path.join(example_data_path, "new_graph.graphml.xml"))

accessibility_matrix = getter.get_accessibility_matrix(transport_graph)

In [18]:
accessibility_matrix.df.head()

id,0,1,2,3,4,5,6,7,8,9,...,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.1,8.2,0.1,8.2,8.2,5.5,3.9,3.9,13.2,...,22.2,19.0,17.4,17.4,16.7,19.4,19.5,20.3,20.1,19.2
1,0.1,0.0,8.1,0.0,8.1,8.1,5.6,4.0,4.0,13.3,...,22.3,19.1,17.5,17.5,16.8,19.5,19.6,20.4,20.2,19.3
2,8.2,8.1,0.0,8.1,0.0,0.0,13.6,11.6,11.6,9.0,...,22.6,19.4,18.3,18.3,18.0,22.5,19.5,20.3,22.4,20.4
3,0.1,0.0,8.1,0.0,8.1,8.1,5.6,4.0,4.0,13.3,...,22.3,19.1,17.5,17.5,16.8,19.5,19.6,20.4,20.2,19.3
4,8.2,8.1,0.0,8.1,0.0,0.0,13.6,11.6,11.6,9.0,...,22.6,19.4,18.3,18.3,18.0,22.5,19.5,20.3,22.4,20.4


Save result for the next example

In [16]:
aggregated_blocks.to_gdf().to_parquet(os.path.join(example_data_path, "data_getter_blocks.parquet"))
# accessibility_matrix.df.to_pickle(os.path.join(example_data_path, "data_getter_matrix.pickle"))